<a href="https://colab.research.google.com/github/swhyuni/hacktiv8-exercises/blob/main/h8dsft_P0W3_sri_wahyuni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Graded Challenge 3
Nama  : sri wahyuni
Batch : RMT 16

Authenticate to GCP

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Connect to the Bigquery API

In [2]:
from google.cloud import bigquery
project_id = "hacktiv8-yuni"
client = bigquery.Client(project=project_id)

In [9]:
# 1. Jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022

jumlah_transaksi = client.query('''
SELECT extract(year from created_at) as year, extract(month from created_at) as month, count(created_at) as jumlah_transaksi
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status='Complete' and created_at BETWEEN '2022-01-01' AND '2022-09-30'
GROUP BY year,month
ORDER BY year,month ASC
''').to_dataframe()
print(jumlah_transaksi)

   year  month  jumlah_transaksi
0  2022      1              1555
1  2022      2              1458
2  2022      3              1896
3  2022      4              2002
4  2022      5              2142
5  2022      6              2202
6  2022      7              2711
7  2022      8              3146
8  2022      9              3392


Insight : Jumlah transaksi yang berstatus complete tiap bulan selama Q1 sampai Q3 (Januari - September) 2022 pada bulan februari mengalami penurusan sebesar 6,23% namun pada bulan selanjutnya mengalami kenaikan jumlah transaksi secara terus menerus

In [10]:
# 2. Total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022

total_penjualan = client.query('''
SELECT extract(year from created_at) as year, extract(month from created_at) as month, sum(sale_price) as total_penjualan
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status='Complete' and created_at BETWEEN '2022-01-01' AND '2022-09-30'
GROUP BY year,month
ORDER BY year,month ASC
''').to_dataframe()
print(total_penjualan)

   year  month  total_penjualan
0  2022      1     92439.170170
1  2022      2     83947.680040
2  2022      3    113608.770018
3  2022      4    121271.810110
4  2022      5    124586.960032
5  2022      6    130889.060151
6  2022      7    164954.250109
7  2022      8    187644.250148
8  2022      9    200580.560122


Total penjualan pada Q1 - Q3 (Januari - September) 2022 berbanding lurus dengan jumlah transaksi dimana pada bulan februari dikarenakan transaksi menurun makan total penjualan menurun dari 92439.17 USD menurun menjadi 83947.68 USD atau 9,18%. Bulan selanjutnya total penjualan sama meningkanya dengan jumlah transaksi.

In [13]:
# 3. user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022 
jumlah_user = client.query('''
SELECT extract(year from created_at) as year, extract(month from created_at) as month, count(user_id) as jumlah_user
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status='Complete' and created_at BETWEEN '2022-01-01' AND '2022-09-30'
GROUP BY year,month
ORDER BY year,month ASC
''').to_dataframe()

print(jumlah_user)

   year  month  jumlah_user
0  2022      1         1555
1  2022      2         1458
2  2022      3         1896
3  2022      4         2002
4  2022      5         2142
5  2022      6         2202
6  2022      7         2711
7  2022      8         3146
8  2022      9         3392


Insight : transaksi yang berhasil / complete mengalami peningkatan secara signifikan mulai dari bulan maret hingga bulan september atau akhir quartal 3, hanya ada penurunan pendapatan pada bulan februari saja

In [18]:
# 4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022?
client.query('''
SELECT tahun, bulan, kategori, total_order
FROM (SELECT *,
      DENSE_RANK() OVER (PARTITION BY tahun, bulan ORDER BY total_order DESC) as rank
      FROM (SELECT EXTRACT(YEAR FROM created_at) AS tahun,
            EXTRACT(MONTH FROM created_at) AS bulan,
            b.category AS kategori,
            COUNT(b.category) AS total_order
        FROM bigquery-public-data.thelook_ecommerce.order_items AS a
        INNER JOIN bigquery-public-data.thelook_ecommerce.products AS b
        ON a.product_id=b.id
        WHERE a.status='Complete'
        AND a.created_at BETWEEN "2022-01-01" AND "2022-09-30"
        GROUP BY tahun, bulan, b.category)
        )
        WHERE rank=1
        ORDER BY tahun, bulan
        ''').to_dataframe()

,tahun,bulan,kategori,total_order
0,2022,1,Intimates,119
1,2022,2,Intimates,122
2,2022,3,Jeans,140
3,2022,4,Intimates,151
4,2022,5,Intimates,162
5,2022,6,Intimates,182
6,2022,7,Jeans,200
7,2022,8,Intimates,232
8,2022,9,Intimates,267


Insight: produk kategori intimates menjadi kategori produk terbanyak dibeli oleh customer, dari 9 bulan data yang diolah 7 bulan diantaranya adalah produk kategori intimates dan tertinggi kedua adalah kategori jeans.

In [19]:
# 5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022?
client.query('''
SELECT tahun, bulan, kategori, jumlah_pendapatan
FROM (SELECT *,
      DENSE_RANK() OVER (PARTITION BY tahun, bulan ORDER BY jumlah_pendapatan DESC) as rank
      FROM (SELECT EXTRACT(YEAR FROM created_at) AS tahun,
            EXTRACT(MONTH FROM created_at) AS bulan,
            b.category AS kategori,
            SUM(sale_price) AS jumlah_pendapatan
        FROM bigquery-public-data.thelook_ecommerce.order_items AS a
        INNER JOIN bigquery-public-data.thelook_ecommerce.products AS b
        ON a.product_id=b.id
        WHERE a.status='Complete'
        AND a.created_at BETWEEN "2022-01-01" AND "2022-09-30"
        GROUP BY tahun, bulan, b.category)
        )
        WHERE rank=1
        ORDER BY tahun, bulan
''').to_dataframe()


,tahun,bulan,kategori,jumlah_pendapatan
0,2022,1,Outerwear & Coats,11693.020027
1,2022,2,Outerwear & Coats,10792.279949
2,2022,3,Jeans,14267.360010
3,2022,4,Outerwear & Coats,14542.200010
4,2022,5,Jeans,15153.360016
5,2022,6,Jeans,15990.480015
6,2022,7,Outerwear & Coats,22072.299984
7,2022,8,Outerwear & Coats,23969.779934
8,2022,9,Outerwear & Coats,28793.159983


Insigth : produk kategori intimates menjadi produk terbanyak di beli oleh customer tapi tidak menjadi produk yang tertinggi berdasarkan pendapatannya. namun yang menjadi produk yang memiliki pendapatan terbesar adalah outerwear & coats yaitu selama 5 bulan di bulan januari, februari, april, juli, agustus dan september. pada bulan maret, mei dan juni produk tertinggi jumlah pendapatannya adalah jeans

In [32]:
#6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)
df = client.query('''
WITH
  first_orders AS (
  SELECT
    a.order_id,
    a.user_id,
    ROW_NUMBER() OVER(PARTITION BY a.user_id ORDER BY a.created_at ASC) order_sequence
  FROM
    `bigquery-public-data.thelook_ecommerce.orders` a
  WHERE
    a.status = 'Complete' AND a.created_at BETWEEN "2022-01-01" AND "2022-09-30" QUALIFY order_sequence = 1 )
SELECT
  COUNT(DISTINCT b.order_id) AS total_transaksi,
  SUM(b.sale_price) total_penjualan,
  COUNT(DISTINCT fo.user_id) total_user
FROM
  first_orders fo
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.order_items` b
ON
  b.order_id = fo.order_id
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  p.id = b.product_id
GROUP BY
  p.category
ORDER BY
  2 desc
''').to_dataframe()
df


,total_transaksi,total_penjualan,total_user
0,946,142827.919847,946
1,1293,132652.280198,1293
2,1098,83931.779988,1098
3,1199,70655.080038,1199
4,498,66533.949784,498
5,1172,65700.570031,1172
6,1136,58527.650134,1136
7,1132,55418.920177,1132
8,1231,53433.420146,1231
9,551,50457.040104,551


In [33]:
# korelasi dengan pandas
import pandas as pd
import numpy as np
df.corr().style.background_gradient(cmap='coolwarm')

,total_transaksi,total_penjualan,total_user
total_transaksi,1.000000,0.704097,1.000000
total_penjualan,0.704097,1.000000,0.704097
total_user,1.000000,0.704097,1.000000


Diantara total_transaksi, total_penjualan dan total_user memiliki keterkaitan namun total_transaksi sangat terkait dengan total_user karena disetiap transaksi akan ada data user yang melakukan transaksi. sedangkan setiap user pasti memiliki total penjualan yang berbeda dengan user lainnya.

### Problem Statement

Meningkatkan pendapatan dengan membuat kampanye berdasarkan produk yang paling diminati pada kuartal 4

POIN ANALISIS

1. Kesimpulan:
Berdasarkan problem statement yang dibuat dengan informasi data yang diperoleh bahwa:
- Performance thelook e-commerce meningkat secara signifikan di kuartal 1 sampai kuartal 3. pada bulan februari pendapatan menurun sebesar 9.18% namun pada bulan maret pendapatan meningkat dan tidak pernah mengalami penurunan hingga akhir kuartal 3 (september).
- Produk kategori intimate berkontribusi hampir setiap bulan menjadi produk terbanyak di beli, namun hal tersebut **tidak** membuat produk intimate menjadi produk yang penghasilan pendapatan terbesar. hal tersebut bisa dipengaruhi dengan berbagai kemungkinan salah satunya harga produk tersebut. 
- produk kategori outerwear & coats menjadi produk dengan penghasilan terbesar di setiap bulannya 
- jika peruhaan ingin meningkatkan jumlah pendapatan makan bisa memfokuskan produk kategori intimate dan outerwear & coats atau salah satu diantaranya

POIN PERTANYAAN

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
- berdasarkan problem statement yang dibuat dapat terukur, namun untuk menambah informasi bisa menambahkan/mencari data lain seperti, user dengan rata-rata transaksi terbesar, jumlah user yang melakukan pembelian ulang, Mencari pembeli high value.
2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

- target pendapatan awal kuartal 4 sebesar $250000 (24,64%) sulit untuk dicapai jika dilihat dari persetase peningkatan pendapatan rata-rata dari Q1-Q3 adalah 12,85%

3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

- berdasarkan trend transaksi dan jumlah user ratarata setiap bulannya mengalami kenaikan yaitu pada awal kuartal 1 sebanyak 1555 dan pada akhir kuartal 3 sebanyak 3392 atau 54,17%. karena jumlah transaksi dan user yang meningkat maka pendapatan ikut meningkat yaitu awal kuartal 1 sebesar $92.439 dan pada akhir kuartal 3 sebesar $200.581 atau 53,91% dan jika dirata-ratakan pendapatan tiap bulan meningkat sebesar 12,85% hal ini membuat thelook e-commerce layak untuk mendapatkan investor